In [2]:
!pip install transformers torch librosa soundfile  datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
# mount google drive to fetch the relevant files

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import warnings
warnings.filterwarnings('ignore')
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperFeatureExtractor
import librosa
import torch

# Load model and processor from the specified folder
model_folder_path = "/content/drive/MyDrive/whisper-small-en-final_dataset"
model = WhisperForConditionalGeneration.from_pretrained(model_folder_path)

# Initialize processor and feature extractor
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language='English', task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [6]:
# Load audio file
audio_file_path = "/content/download.wav"
# normal_audio_path ="/content/drive/MyDrive/Dataset/MC/MC01/Session2/wav_arrayMic/0118.wav"
audio, sample_rate = librosa.load(audio_file_path, sr=16000)

# Extract audio features
inputs = feature_extractor(audio, sampling_rate=sample_rate, return_tensors="pt")

# Preprocess audio data
inputs = processor.feature_extractor.pad(inputs, return_tensors="pt").input_features[0]

# Initialize model
inputs = torch.tensor(inputs).unsqueeze(0)

# Perform inference
outputs=model.generate(inputs)

# Decode transcribed text
transcribed_text = processor.batch_decode(outputs, skip_special_tokens=True)
print("Transcribed Text:", transcribed_text)

You have passed language=en, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=en.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcribed Text: ['he slowly kicks a slight walk in the open air you gay']


In [10]:
transcription= " ".join(transcribed_text)

In [7]:
# # Decode transcribed text
# transcribed_text = processor.batch_decode(outputs, skip_special_tokens=True)
# print("Transcribed Text:", transcribed_text)

In [11]:
!pip install torch speechbrain librosa soundfile

# Step 2: Import necessary libraries
from speechbrain.pretrained import Tacotron2, HIFIGAN
import torch
import soundfile as sf
from IPython.display import Audio  # Import the Audio class for playback

# Step 3: Load the pretrained models (Tacotron2 and HiFi-GAN)
tacotron2 = Tacotron2.from_hparams(source="speechbrain/tts-tacotron2-ljspeech", savedir="tmpdir_tacotron2", run_opts={"weights_only": True})
vocoder = HIFIGAN.from_hparams(source="speechbrain/tts-hifigan-ljspeech", savedir="tmpdir_vocoder")


# Step 5: Convert the text to mel-spectrogram using Tacotron2
mel_output, mel_length, alignment = tacotron2.encode_text(transcription)

# Step 6: Use HiFi-GAN to convert the mel-spectrogram to a waveform (audio)
waveform = vocoder.decode_batch(mel_output)

# Step 7: Save the waveform to a .wav file
sf.write("generated_speech.wav", waveform.squeeze().cpu().numpy(), 22050)

# Step 8: Play the generated audio
Audio("generated_speech.wav")